For this lab, you will be using the `trip_fare_500k.csv` file found in the `data/nycTaxiData/` folder .

This dataset contains a fairly large number of distinct trips taken in cabs in the NYC area in 2013 (500 thousand of them, to be exact!).

The dataset contains the following information at the top of the file (this is called the header):

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `pickup_datetime`: The time when the ride started
* `payment_type`: How the trip was paid, `UNK` stands for unknown, I have no idea what `NOC` stands for, but lets assume its some known way to pay
* `fare_amount`: Base fare cost of the trip
* `surcharge`: Additional charges that are not tolls
* `mta_tax`: The mta has to get its cut, right? :)
* `tip_amount`: How generous the rider(s) decided to be
* `tolls_amount`: How much money you had to pay in tolls
* `total_amount`: How much the trip cost, all in

Phew, That's a lot of info!

Here is the assignment:

1. What was the most expensive/least expensive trip taken?
* Does the overall `total_amount` paid per ride correlate with `tip_amount` per ride?
* Does it correlate when you remove all rides with unknown `payment_type`?
* Calculate the average cost of a trip in this dataset given the following conditions:
  1. Across the whole dataset
  2. Across the whole dataset when the `payment_type` is known (not `UNK`)
  3. For each `payment_type`
  4. Which `payment_type` had the highest average cost?
  5. Which `payment_type` had the largest spread in how much people paid (largest standard deviation)?
  6. Which `payment_type` had the most generous people (had the highest average tip), including unknown payment types?
  7. What hour in the day were people most generous, on average, when they got into a cab?
  8. What hour of the day did people fluctuate the most in terms of tips? That is, do some hours lead to unpredictable tip amounts? 
* Which person (`hack_license`) made the most money:
  1. In total
  2. On a per-trip basis, given that they took at least 20 trips
* Does the number of trips a given cabbie takes (her/his experience) correlate with how well she/he is tipped? If so, in what direction?
* Does the number of times a given cab is used correlate with how well the person driving the cab is tipped? That is, are there "lucky" cabs?
* Which `vendor_id` had the higher average `surcharge` on a per-hour basis?
* Which hour in the day: 
  1. Did people most frequently take rides?
  2. Did people least frequently take rides?
  3. Had the largest number of unique cabs on the street?
  4. Had the least number of cabs in the street?
  5. What is the average number of cabs on the streets in NYC in each quarter of the day (at least in this dataset?)?

Use the rest of this notebook to work through all these questions. 

I will be coming around to everyone to help/guide you in your data science quest!

If you can tackle all of these questions, then you've learned a lot already! 

If not, don't worry, this stuff is hard and I will gladly help/guide you through all of this.

but take charge of your learning!

This means:

* Ask a neighbor to help if you don't understand something. 
* If your neighbor can't help you, try using:
  * the interactive documentation I showed you how to use earlier
  * [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/index.html)
  * [google](http://www.google.com)
  * [stackoverflow](http://stackoverflow.com) to see if someone in the internet ether has had a similar problem before
  * if none of this works, then I will gladly help you
* This will accomplish at least two things:
  * It will get you to use online resources and take charge of your learning
  * Get you to learn alternative approaches (those I did not show you today) to solving your problem

I've started the bare-bones script for you by:

* importing what I ~~think~~ know you will need
* loading the dataset into a variable called `fareData` that stores the data as a `DataFrame` object (you might need to change the path to where the file is located on your system)
* formatting the timestamp for you so that you don't have to figure out how to do it, because spending 30+ minutes  (or more) trying to figure it out is not the point of the assignment. This way, all of the functions in `fareData.pickup_datetime.dt` can immediately be used on the `pickup_datetime` column your dataset.

The rest I leave to you. Happy hacking!

In [1]:
import pandas as pd
import numpy as np

In [3]:
fareData = pd.read_csv("nycTaxiData/trip_fare_500k.csv")
fareData.pickup_datetime = pd.to_datetime(fareData.pickup_datetime,format="%Y-%m-%d %H:%M:%S")
fareData.dtypes #this is to confirm that the pickup_datetime column, as well as all of the other
# columns, are in the appropriate formats (pickup_datetime should be in datetime64 format)
# if it isn't something is wrong, and we need to figure what that is

FileNotFoundError: File b'nycTaxiData/trip_fare_500k.csv' does not exist